In [ ]:
# dataframe and math
import pandas as pd
import numpy as np

# random module
import random

# visuals
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
PATHS = {
    'train': '../input/project2/train.csv/train.csv',
    'test': '../input/project2/train.csv/test.csv',
    'clean_train': "./clean_train.csv"
}

DIVISION DE DATASETS

In [ ]:
clean_df = pd.read_csv(
    PATHS['clean_train']
)

In [ ]:
clean_df.info()

In [ ]:
TO_STRING = clean_df.columns[clean_df.dtypes.eq('object')]

In [ ]:
for col in TO_STRING:
    clean_df[col] = clean_df[col].astype("category")

In [ ]:
clean_df.info()

X & Y

In [ ]:
target = clean_df['HasDetections']

In [ ]:
features = clean_df.drop('HasDetections', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
sample_ratio = 0.25

In [ ]:
_, X_sample, _, y_sample = train_test_split(features, target, test_size = sample_ratio)

In [ ]:
print("Se trabajará con una muestra del %.0f%% de los datos originales" % (len(X_sample) / len(clean_df) * 100))

In [ ]:
## Revisamos el balance de los datos
y_sample.value_counts()

SCALING

In [ ]:
df_columns = X_sample.columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
mm_scaler = MinMaxScaler()

In [ ]:
s_scaler = StandardScaler()

In [ ]:
df_scaled = s_scaler.fit_transform(X_sample)
df_scaled = pd.DataFrame(df_scaled, columns=df_columns)

MODELS

In [ ]:
## import for evaluations
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, mean_squared_error, precision_recall_curve
from scikitplot.metrics import plot_roc

In [ ]:
## function for plotting confusion matrix in a more comprehensive format
def plot_confusion_matrix(matrix, title):
    group_names = ['Real Negativo','Falso Positivo','Falso Negativo','Real Positivo']
    group_counts = ["{0:0.0f}".format(value) for value in matrix.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in matrix.flatten() / np.sum(matrix)]

    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)

    ax = sns.heatmap(matrix, annot=labels, fmt='', cmap='BuPu')

    ax.set_title('Matriz de Confusión de {}\n'.format(title));

    ax.xaxis.set_ticklabels(['No Detections','Detections'])
    ax.yaxis.set_ticklabels(['No Detections','Detections'])
    ax.set_xlabel('Valores Predichos');
    ax.set_ylabel('Valores Correctos ');
    
    plt.show()

In [ ]:
test_ratio = 0.3

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size = test_ratio)

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [ ]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'binary:logistic',
    'eval_metric':'auc'
}

num_boost_round = 999

In [ ]:
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

In [ ]:
xgb_preds = xgb_model.predict(dtest)

binary_preds = [0 if x < 0.5 else 1 for x in xgb_preds]

In [ ]:
xgb_cm = confusion_matrix(y_test, binary_preds)

In [ ]:
plot_confusion_matrix(xgb_cm, "XG Boost")

In [ ]:
print(classification_report(y_test, binary_preds))